# Compare existing results

This sheet enables you to compare results that have already been generated. It includes functions to plot cell number in each generation for each condition, along with plots that indicate whether a combination of effects is greater than expected if they were additive.

In [ ]:
using Plots 
using CSV
using Tables

In [ ]:
function plotComparativeLineageTrees(treeFileName,cellCountFileName,thisColorPalette)

    graphingArray=CSV.File(treeFileName,types=Float64, silencewarnings=true) |> Tables.matrix
    
    cellCountArray=zeros(maxGens,Int64(maxTimeTC))


    #first we need to sort by number of generations reached
    graphingArray=graphingArray[sortperm(graphingArray[:, end]), :]

    nextYCoord=0
    #now we need to go through each
    p1=plot(xlabel="",xlim=(0,maxTimeTC),xticks = (0:60*24:maxTimeTC, string.(0:24:Int64(maxTimeTC))),legend=false)
    for i in 1:size(graphingArray,1)
        thisCellMaxGen=Int64(graphingArray[i,end])
        cellsInThisCellMaxGen=2^(thisCellMaxGen)-1
        #nextYCoord=nextYCoord+cellsInThisCellMaxGen
        nextYCoord=nextYCoord+cellsInThisCellMaxGen
        middle=nextYCoord+1+cellsInThisCellMaxGen/2
        lastGenEnd=1
        nextGenMiddles=[]
        for thisGen in 1:thisCellMaxGen
            cellsInThisGen=2^(thisGen-1)
            stepSizePerCell=cellsInThisCellMaxGen/(cellsInThisGen-1)
            thisGenFateTime=graphingArray[i,thisGen]
            plottingArrayX=[NaN,NaN]
            plottingArrayY=[NaN,NaN]
            thisGenMiddles=copy(nextGenMiddles)
            nextGenMiddles=[]
            cellCountArray[thisGen,Int64(round(lastGenEnd)):Int64(round(thisGenFateTime))-1]=cellCountArray[thisGen,Int64(round(lastGenEnd)):Int64(round(thisGenFateTime))-1].+cellsInThisGen
            for cellIndex in 1:(2^(thisGen-1))
                if thisGen==1
                    thisYCoord=middle
                else
                    #thisYCoord=middle-((cellsInThisGen/2)*stepSize)+(stepSize*(cellIndex-1))
                    thisYCoord=thisGenMiddles[cellIndex]
                end
                append!(plottingArrayX,[lastGenEnd,thisGenFateTime])
                append!(plottingArrayY,[thisYCoord,thisYCoord])
                append!(plottingArrayX,[NaN,NaN])
                append!(plottingArrayY,[NaN,NaN])
                if(thisGen==thisCellMaxGen)
                    #plot an x
                    append!(plottingArrayX,[thisGenFateTime-(10*crossSize),thisGenFateTime+(10*crossSize)])
                    append!(plottingArrayY,[thisYCoord+crossSize,thisYCoord-crossSize])
                    append!(plottingArrayX,[NaN,NaN])
                    append!(plottingArrayY,[NaN,NaN])
                    append!(plottingArrayX,[thisGenFateTime-(10*crossSize),thisGenFateTime+(10*crossSize)])
                    append!(plottingArrayY,[thisYCoord-crossSize,thisYCoord+crossSize])
                    append!(plottingArrayX,[NaN,NaN])
                    append!(plottingArrayY,[NaN,NaN])
                else
                    #plot the bar between related cells
                    append!(plottingArrayX,[thisGenFateTime,thisGenFateTime])
                    append!(plottingArrayY,[thisYCoord-((cellsInThisCellMaxGen/2)/(cellsInThisGen)),thisYCoord+((cellsInThisCellMaxGen/2)/cellsInThisGen)])
                    append!(plottingArrayX,[NaN,NaN])
                    append!(plottingArrayY,[NaN,NaN])
                    append!(nextGenMiddles,[thisYCoord-((cellsInThisCellMaxGen/2)/(cellsInThisGen))])
                    append!(nextGenMiddles,[thisYCoord+((cellsInThisCellMaxGen/2)/(cellsInThisGen))])
                end
            end
            plot!(p1,plottingArrayX,plottingArrayY,linecolor=thisColorPalette[thisGen])
            lastGenEnd=graphingArray[i,thisGen]
        end  
        nextYCoord=nextYCoord+maximum([cellsInThisCellMaxGen,10])
    end
    display(p1)
    cellCountArray
    p2=plot()
    totalCount=sum(cellCountArray,dims=1)
    for thisGen in 1:maxGens
        thisGenCells=cellCountArray[thisGen,:]
        plot!(p2,1:maxTimeTC,thisGenCells,linecolor = thisColorPalette[thisGen],label="gen "*string(thisGen))
        
    end
    plot!(p2,[1:maxTimeTC],totalCount',linecolor = :black,label="total",xlim=(0,maxTimeTC),xticks = (0:60*24:maxTimeTC, string.(0:24:Int64(maxTimeTC))))
    display(p2)

    
    p3=plot()
    
    generationArray=zeros(maxGens,size(cellCountArray,2))
    for thisGen in 1:maxGens
        thisGenCells=zeros(size(cellCountArray))
        thisGenCounts=cellCountArray[thisGen,:]
        for addingGen in thisGen:maxGens
            generationArray[addingGen,:]=generationArray[addingGen,:]+thisGenCounts
        end
        plot!(p3,1:maxTimeTC,generationArray[thisGen,:][1:Int64(maxTimeTC)],linecolor = thisColorPalette[thisGen])
    end
    plot!(p3,ylabel="cells in gen",labels=string.([1:maxGens]),xlim=(0,maxTimeTC),xticks = (0:60*24:maxTimeTC, string.(0:24:Int64(maxTimeTC))))
    plot!(p3,[1:maxTimeTC],totalCount',linecolor = :black,label="total")

    p3=current()

    
    IJulia.clear_output(true)
    thisPlot=plot(p1,p2,p3,legend = false,size=(900,min(2000,max(500,nextYCoord*5))),layout = grid(3, 1,heights=[0.8,0.1,0.1]))
    display(thisPlot)

end

In [ ]:
# palettes=[:acton,:bamako,:lapaz]
# overallDateStamp="2022-03-30T08:18:13.479"
# filesInDir=conditions
# p2=plot(size=(900,250),ylabel="cell count")

# for folder in filesInDir
#     thisCondition=folder
#     data = readdlm(folderToPlot*"/"*thisCondition*"/cellCounts.csv", ',', Float64)
#     total=sum(data[:,1:end].>0,dims=1)
#     p2=plot!(1:size(total,2),total',label=thisCondition,xlim=(0,maxTimeTC),xticks = (0:60*24:maxTimeTC, string.(0:24:Int64(maxTimeTC))))
#     IJulia.clear_output(true)
#     display(p2)
# end

# crossSize=5;
# folderToPlot="NASoutputs/"*overallDateStamp
# conds
# #folderToPlot="NASoutputs/2022-02-08T12:31:21.091"
# pathToPlot=folderToPlot*"/"*condToPlot*"/"

# #define some files to plot
# #treeFileName="graphStructure.csv"
# #cellCountFileName="cellCounts.csv"
# treeArrayFile=pathToPlot*"treeStructure.csv"
# cellCountFileName=pathToPlot*"cellCounts.csv"


In [ ]:
function compareCellNumbers(conditions,palettes)

    p3=plot(ylabel="cells in gen",xlabel="time (h)",labels=string.([1:maxGens]),xlim=(0,maxTimeTC),xticks = (0:60*24:maxTimeTC, string.(0:24:Int64(maxTimeTC))))
    WTorder=nothing
    condIndex=1
    WTyIndexes=[]
    thisCondScalar=0;
    for condToPlot in conditions

        thisColorPalette=palette(palettes[Int64(condIndex)],Int64(maxGens))
        crossSize=5;
        folderToPlot="SSDoutputs/"*overallDateStamp
        #folderToPlot="NASoutputs/2022-02-08T12:31:21.091"
        pathToPlot=folderToPlot*"/"*condToPlot*"/"

        #define some files to plot
        #treeFileName="graphStructure.csv"
        #cellCountFileName="cellCounts.csv"
        treeArrayFile=pathToPlot*"treeStructure.csv"
        cellCountFileName=pathToPlot*"cellCounts.csv"
        graphingArray=CSV.File(treeArrayFile,types=Float64, silencewarnings=true) |> Tables.matrix

        cellCountArray=zeros(maxGens,Int64(maxTimeTC))


        #first we need to sort by number of generations reached
        order=sortperm(graphingArray[:, end])
        if(condIndex==1)
            WTorder=order
            graphingArray=graphingArray[order, :]
        else
            graphingArray=graphingArray[WTorder, :]
        end

        nextYCoord=0
        #now we need to go through each
        for i in 1:size(graphingArray,1)
            thisCellMaxGen=Int64(graphingArray[i,end])
            cellsInThisCellMaxGen=2^(thisCellMaxGen)-1
            #nextYCoord=nextYCoord+cellsInThisCellMaxGen
            nextYCoord=nextYCoord+cellsInThisCellMaxGen
            middle=nextYCoord+1+cellsInThisCellMaxGen/2
            if(condIndex==1)
                append!(WTyIndexes,middle+10)
            else
                middle=WTyIndexes[i]
            end
            lastGenEnd=1
            nextGenMiddles=[]
            for thisGen in 1:thisCellMaxGen
                cellsInThisGen=2^(thisGen-1)
                stepSizePerCell=cellsInThisCellMaxGen/(cellsInThisGen-1)
                thisGenFateTime=graphingArray[i,thisGen]
                thisGenMiddles=copy(nextGenMiddles)
                cellCountArray[thisGen,Int64(round(lastGenEnd)):Int64(round(thisGenFateTime))-1]=cellCountArray[thisGen,Int64(round(lastGenEnd)):Int64(round(thisGenFateTime))-1].+cellsInThisGen
                lastGenEnd=graphingArray[i,thisGen]
            end  
            nextYCoord=nextYCoord+maximum([cellsInThisCellMaxGen,10])
        end
        totalCount=sum(cellCountArray,dims=1)
        generationArray=zeros(maxGens,size(cellCountArray,2))
        for thisGen in 1:maxGens
            thisGenCells=zeros(size(cellCountArray))
            thisGenCounts=cellCountArray[thisGen,:]
            for addingGen in thisGen:maxGens
                generationArray[addingGen,:]=generationArray[addingGen,:]+thisGenCounts
            end
            plot!(p3,1:maxTimeTC,generationArray[thisGen,:][1:Int64(maxTimeTC)],linecolor = thisColorPalette[thisGen],label=false)
        end
        plot!(p3,[1:maxTimeTC],totalCount',linecolor =thisColorPalette[1] ,label=condToPlot,linewidth=3)



        IJulia.clear_output(true)
        thisPlot=plot(p3,legend = true,size=(900,400),layout = grid(3, 1,heights=[0.8,0.1,0.1]))
        display(thisPlot)
        condIndex=condIndex+1
    end
    thisCondScalar+=500;
end

In [ ]:
function compareCellNumbersRelative(conditions,palettes)

    p3=plot(title="cell number - WT cell number",ylabel="cells > WT",labels=string.([1:maxGens]),xlim=(0,maxTimeTC),xticks = (0:60*24:maxTimeTC, string.(0:24:Int64(maxTimeTC))))
    WTorder=nothing
    condIndex=1
    WTyIndexes=[]
    thisCondScalar=0;
    WTArrayCellCountArray=[]
    synergyArray=[]

    for condToPlot in conditions

        thisColorPalette=palette(palettes[Int64(condIndex)],Int64(maxGens))
        crossSize=5;
        folderToPlot="SSDoutputs/"*overallDateStamp
        #folderToPlot="NASoutputs/2022-02-08T12:31:21.091"
        pathToPlot=folderToPlot*"/"*condToPlot*"/"

        #define some files to plot
        #treeFileName="graphStructure.csv"
        #cellCountFileName="cellCounts.csv"
        treeArrayFile=pathToPlot*"treeStructure.csv"
        cellCountFileName=pathToPlot*"cellCounts.csv"
        graphingArray=CSV.File(treeArrayFile,types=Float64, silencewarnings=true) |> Tables.matrix

        cellCountArray=zeros(maxGens,Int64(maxTimeTC))


        #first we need to sort by number of generations reached
        order=sortperm(graphingArray[:, end])
        if(condIndex==1)
            WTorder=order
            graphingArray=graphingArray[order, :]
        else
            graphingArray=graphingArray[WTorder, :]
        end

        nextYCoord=0
        #now we need to go through each
        for i in 1:size(graphingArray,1)
            thisCellMaxGen=Int64(graphingArray[i,end])
            cellsInThisCellMaxGen=2^(thisCellMaxGen)-1
            #nextYCoord=nextYCoord+cellsInThisCellMaxGen
            nextYCoord=nextYCoord+cellsInThisCellMaxGen
            middle=nextYCoord+1+cellsInThisCellMaxGen/2
            if(condIndex==1)
                append!(WTyIndexes,middle+10)
            else
                middle=WTyIndexes[i]
            end
            lastGenEnd=1
            nextGenMiddles=[]
            for thisGen in 1:thisCellMaxGen
                cellsInThisGen=2^(thisGen-1)
                stepSizePerCell=cellsInThisCellMaxGen/(cellsInThisGen-1)
                thisGenFateTime=graphingArray[i,thisGen]
                thisGenMiddles=copy(nextGenMiddles)
                cellCountArray[thisGen,Int64(round(lastGenEnd)):Int64(round(thisGenFateTime))-1]=cellCountArray[thisGen,Int64(round(lastGenEnd)):Int64(round(thisGenFateTime))-1].+cellsInThisGen
                lastGenEnd=graphingArray[i,thisGen]
            end  
            nextYCoord=nextYCoord+maximum([cellsInThisCellMaxGen,10])
        end
        
        if condIndex==1
            WTArrayCellCountArray=copy(cellCountArray)
            #println(WTArrayCellCountArray)
            cellCountArray.=0
            synergyArray=zeros(3,Int64(maxTimeTC))
        else
            cellCountArray=cellCountArray.-WTArrayCellCountArray
        end
        totalCount=sum(cellCountArray,dims=1)
        if condIndex>1
           synergyArray[condIndex-1,:]=totalCount
        end
        generationArray=zeros(maxGens,size(cellCountArray,2))
        for thisGen in 1:maxGens
            thisGenCells=zeros(size(cellCountArray))
            thisGenCounts=cellCountArray[thisGen,:]
            for addingGen in thisGen:maxGens
                generationArray[addingGen,:]=generationArray[addingGen,:]+thisGenCounts
            end
            #comment out this line if you don't want individual generations to be plotted
            plot!(p3,1:maxTimeTC,generationArray[thisGen,:][1:Int64(maxTimeTC)],linecolor = thisColorPalette[thisGen],label=false)
        end
        plot!(p3,[1:maxTimeTC],totalCount',linecolor =thisColorPalette[1] ,label=condToPlot,linewidth=3)

        
        #println(totalCount)
        #println(condToPlot)
        plot!(p3,legend = true,size=(900,400),layout = grid(3, 1,heights=[0.8,0.1,0.1]))
        #display(thisPlot)

        condIndex=condIndex+1
        
    end
    expectedIfAdditive=synergyArray[1,:].+synergyArray[2,:]
    actualResult=synergyArray[3,:]
    #p4=plot([1:maxTimeTC],actualResult,linecolor =:red ,label="actual",linewidth=2,fill=(0,0.5,:red))
    #plot!(p4,[1:maxTimeTC],expectedIfAdditive,linecolor =:black ,label="additive",linewidth=1,fill=(0,1.0,:white))
    #plot!(p4,legend = true,size=(900,400),layout = grid(3, 1,heights=[0.8,0.1,0.1]),ylabel="cells in gen",xlabel="time (h)",labels=string.([1:maxGens]),xlim=(0,maxTimeTC),xticks = (0:60*24:maxTimeTC, string.(0:24:Int64(maxTimeTC))))
    lower=minimum(hcat(actualResult,expectedIfAdditive),dims=2)
    upper=maximum(hcat(actualResult,expectedIfAdditive),dims=2)
    p5=plot([1:maxTimeTC], [actualResult actualResult], fillrange=[lower, upper], fillalpha=0.35, c = [:red :blue], label = [">additive" "<additive"], legend = :topleft)
    plot!(p5,size=(900,400),layout = grid(3, 1,heights=[0.8,0.1,0.1]),title="expected if additive vs actual combo",ylabel="cell number over WT",labels=string.([1:maxGens]),xlim=(0,maxTimeTC),xticks = (0:60*24:maxTimeTC, string.(0:24:Int64(maxTimeTC))))
    differenceFromExpected=actualResult.-expectedIfAdditive
    p6=plot([1:maxTimeTC], differenceFromExpected)
    plot!(p6,size=(900,400),layout = grid(3, 1,heights=[0.8,0.1,0.1]),title="[actual combo] - [additive prediction]",ylabel="cell number over WT",xlabel="time (h)",labels=string.([1:maxGens]),xlim=(0,maxTimeTC),xticks = (0:60*24:maxTimeTC, string.(0:24:Int64(maxTimeTC))),legend=true)
    thisPlot=plot(p3,p5,p6,size=(900,900),layout = grid(3, 1,heights=[0.3,0.3,0.3]))
    display(thisPlot)
end

In [ ]:
overallDateStamp="2022-03-31T14:05:51.606"
plotly()
maxGens=8 # maximum number of gens in total
palettes=[[:blue,:gray],[:black,:gray],[:red,:gray],[:green,:gray],:lapaz]
global maxTimeTC=float(60*120) # how long should the entire simulation run

## Explanation of plots

1. Cell number and the number of cells in each generation for each condition
2. Each condition subtracted from the WT cell number to indicate whether cell number has increased or decrease. Each generation is shown so this is likely messy but commenting out line 84 above will hide these generation lines.
3. This plot shows the expected effect if condition 2 and 3 were additive, along with the actual simulated effect of the two conditions combined. The region is shaded based on whether the observed effect is greater or lower than additive.
4. This plot just shows the absolute change in cell number of the combintion minus the change in cell number if the combination was additive. This is designed to show you which parts of the time course greater than additive effects are seen.

In [ ]:
thisPlot=["WT","cMyc1p5","BCL21p5","DH1p5"]
compareCellNumbers(thisPlot,palettes)
compareCellNumbersRelative(thisPlot,palettes)

In [ ]:
thisPlot=["WT","cMyc2","BCL22","DH2"]
compareCellNumbers(thisPlot,palettes)
compareCellNumbersRelative(thisPlot,palettes)

In [ ]:
overallDateStamp="2022-04-01T12:39:04.116"
thisPlot=["WT","BCL21p5","CKS1B1p5","gain1q1p5"]
compareCellNumbers(thisPlot,palettes)
compareCellNumbersRelative(thisPlot,palettes)

In [ ]:
# thisPlot=["WT","BCL22","CKS1B2","gain1q2"]
# compareCellNumbers(thisPlot,palettes)
# compareCellNumbersRelative(thisPlot,palettes)